In [11]:
import pandas as pd
import quandl

df = quandl.get('WIKI/GOOGL')

print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

Idea of Regression is just to get your data and find a dbest fit line with your data. in y=mx+b we need to find our m and our b. With supervised machine learning it all goes to features and attributes. and finding the relationship between different attributes. 

In [12]:
df = df[["Adj. Open","Adj. High","Adj. Low","Adj. Close","Adj. Volume"]]
df['HL_PCT'] = (df['Adj. High'] - df["Adj. Close"]) / df["Adj. Close"] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df["Adj. Open"]) / df["Adj. Open"] * 100.0

df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  3.712563    0.324968   44659000.0
2004-08-20   54.322689  0.710922    7.227007   22834300.0
2004-08-23   54.869377  3.729433   -1.227880   18256100.0
2004-08-24   52.597363  6.417469   -5.726357   15247300.0
2004-08-25   53.164113  1.886792    1.183658    9188600.0


adjusted close will be a feature. how do we predict the adjuested close for the next day? next five days?

In [15]:
import math

forcast_col = "Adj. Close"
df.fillna(-99999, inplace=True) #we are replacing our na's with outlier data this is a good option for ML

forecast_out = int(math.ceil(0.01*len(df)))
df['label'] = df[forcast_col].shift(-forecast_out)
df.dropna(inplace = True)
print(df.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
Date                                                              
2017-10-06      993.64  0.062397    1.391837    1490744.0  1072.01
2017-10-09      992.31  0.821316   -0.270352    1151035.0  1063.29
2017-10-10      987.80  0.978943   -0.753542    1158864.0  1037.38
2017-10-11     1005.65  0.189927    1.679406    1748443.0  1036.17
2017-10-12     1005.65  0.585691    0.180308    1521137.0  1025.07


so here lets compare the label to the adj. Close

In [20]:
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

x = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

x = preprocessing.scale(x)
df.dropna(inplace = True)
y = np.array(df['label'])

print(len(x), len(y))

3312 3312


In [22]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x,y, test_size = 0.2)
clf = LinearRegression() #can also try svm.SVR()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)
accuracy = clf.score(x_test, y_test)
print(accuracy) #is the squared error
print(forecast_out) #how many days in advance

0.973701576414
34


you want to train and test on different data. with various algorithms you will want to look at the documentation. to see if they can be threaded. (look for number of jobs)